In [20]:
%pip install composeml

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [34]:
import composeml as cp

df = cp.demos.load_transactions()

df[df.columns[:7]].head()

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device
0,298,1,2014-01-01 00:00:00,5,127.64,2,desktop
1,10,1,2014-01-01 00:09:45,5,57.39,2,desktop
2,495,1,2014-01-01 00:14:05,5,69.45,2,desktop
3,460,10,2014-01-01 02:33:50,5,123.19,2,tablet
4,302,10,2014-01-01 02:37:05,5,64.47,2,tablet


In [35]:
df1=df[df.columns[:7]].head()
df1

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device
0,298,1,2014-01-01 00:00:00,5,127.64,2,desktop
1,10,1,2014-01-01 00:09:45,5,57.39,2,desktop
2,495,1,2014-01-01 00:14:05,5,69.45,2,desktop
3,460,10,2014-01-01 02:33:50,5,123.19,2,tablet
4,302,10,2014-01-01 02:37:05,5,64.47,2,tablet


In [44]:
def total_spent(df1):
    total = df1['amount'].sum()
    return total

In [45]:
label_maker = cp.LabelMaker(
    target_dataframe_index="customer_id",
    time_index="transaction_time",
    labeling_function=total_spent,
    window_size="1h",
)

In [50]:
labels = label_maker.search(
    df1.sort_values('transaction_time'),
    num_examples_per_instance=-1,
    gap=1,
    verbose=True,
)

labels.head()

Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| customer_id: 1/1 


,customer_id,time,total_spent
0,2,2014-01-01 00:00:00,254.48
1,2,2014-01-01 00:09:45,126.84
2,2,2014-01-01 00:14:05,69.45
3,2,2014-01-01 02:33:50,187.66
4,2,2014-01-01 02:37:05,64.47


In [51]:

#We can transform the labels by applying threshold to binary labels as follows. 

labels = labels.threshold(200) 

labels.head() 

,customer_id,time,total_spent
0,2,2014-01-01 00:00:00,True
1,2,2014-01-01 00:09:45,False
2,2,2014-01-01 00:14:05,False
3,2,2014-01-01 02:33:50,False
4,2,2014-01-01 02:37:05,False


In [52]:
labels = labels.apply_lead('1h')

labels.head()

,customer_id,time,total_spent
0,2,2013-12-31 23:00:00,True
1,2,2013-12-31 23:09:45,False
2,2,2013-12-31 23:14:05,False
3,2,2014-01-01 01:33:50,False
4,2,2014-01-01 01:37:05,False
